In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset['sex'] = pd.factorize(dataset['sex'])[0]
dataset['smoker'] = pd.factorize(dataset['smoker'])[0]
dataset['region'] = pd.factorize(dataset['region'])[0]
dataset.head()

In [ ]:
train_dataset, test_dataset = np.split(dataset,[int(0.8*len(dataset))])
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
len(test_dataset)

In [ ]:
normalization_layer = layers.experimental.preprocessing.Normalization(axis=-1)
normalization_layer.adapt(np.array(train_dataset))

model = tf.keras.Sequential([
    normalization_layer,
    layers.Dense(3),
    layers.Dense(5),
    layers.Dense(1)
])

In [ ]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.03),
    loss='mae',
    metrics=['mae', 'mse']
)
model.build()
model.summary()

In [ ]:
model.fit(train_dataset, train_labels, epochs=200, batch_size=108, validation_split=0.4)

In [ ]:

loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
